# **DETECTOR DE OBJETOS CON YOLO V11**

In [39]:
# INSTALACIÓN DE LIBRERÍAS NECESARIASA
#pip install ultralytics
#pip install roboflow

In [40]:
# carga de utils
import sys
sys.path.append('../utils')
from utils import generar_lineas_por_manzana, procesar_jsons_en_carpeta, tabulate_jsons_from_folder, copiar_archivos_seleccionados

In [41]:
# carga de librerías
from ultralytics import YOLO
import pandas as pd
import numpy as np
import shutil

In [42]:
# Ruta a la carpeta del dataset
path = '../data/raw/data_set_all_512/'

In [43]:
# Procesamiento de la data de los labels
procesar_jsons_en_carpeta(f'{path}metadata_y_512/','../data/interim_yolo/dataset_labels/')

In [44]:
# Definición de train y test
jsons = tabulate_jsons_from_folder(f'{path}metadata_y_512/')#('../data/raw/data_set_256/metadata_y_256/')

l1 = jsons[jsons['num_polygons_in_window']>=1]['id']
l0 = jsons[jsons['num_polygons_in_window']==0]['id'].sample(n=int(np.round(len(l1)/5)), random_state=42)
l = pd.concat([l1,l0])

l_train = l.sample(frac=0.8, random_state=42)
l_test = l.drop(l_train.index)

In [45]:
copiar_archivos_seleccionados(f'{path}dataset_x_512/', '../data/interim_yolo/train/images/', [str(x)+'.png' for x in l_train])
copiar_archivos_seleccionados(f'{path}dataset_x_512/', '../data/interim_yolo/valid/images/', [str(x)+'.png' for x in l_test])

copiar_archivos_seleccionados('../data/interim_yolo/dataset_labels/', '../data/interim_yolo/train/labels/', [str(x)+'.txt' for x in l_train])
copiar_archivos_seleccionados('../data/interim_yolo/dataset_labels/', '../data/interim_yolo/valid/labels/', [str(x)+'.txt' for x in l_test])

In [46]:
# DESCARGA DE DATASET PERSONALIZADO DESDE ROBOFLOW
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="YP6UX0p6bmRnwAjcSbfY")
#project = rf.workspace("nella-hyakz").project("custom_object_detector_yolo11_v2-rotnm")
#version = project.version(1)
#dataset = version.download("yolov11")
                

In [47]:
# CARGA DEL MODELO BASE YOLO V11
from ultralytics import YOLO
model = YOLO("yolo11s.pt")

In [48]:
# ENTRENAMIENTO DEL MODELO PERSONALIZADO
data_path = "custom_object_detector_yolo11_v2-1/data.yaml"
results= model.train(data=data_path,
epochs=15,
imgsz=512)

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.227  Python-3.11.5 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom_object_detector_yolo11_v2-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train

# HASTA ACÁ ESTÁ BIEN QUE CORRA, PUSHEA EL REPO Y YO ME ENCARGO

# Predicciones

In [49]:
# CARGAR EL MODELO YA ENTRENADO
custom_model = YOLO("runs/detect/train14/weights/best.pt")

In [51]:
# REALIZAR PREDICCIONES SOBRE ALGUNAS IMÁGENES
res = custom_model("custom_object_detector_yolo11_v2-1/testt/images")


image 1/4 c:\Users\JORGE\Documents\ProyectoIntegrador\Block-Detection\Yolo\custom_object_detector_yolo11_v2-1\testt\images\11_jpg.rf.439c99c680c4f6f10d4286abac933be3.jpg: 320x512 45 manzanass, 20.6ms
image 2/4 c:\Users\JORGE\Documents\ProyectoIntegrador\Block-Detection\Yolo\custom_object_detector_yolo11_v2-1\testt\images\14_jpg.rf.922b4601bd4299301469184f0b8de8e3.jpg: 320x512 53 manzanass, 9.5ms
image 3/4 c:\Users\JORGE\Documents\ProyectoIntegrador\Block-Detection\Yolo\custom_object_detector_yolo11_v2-1\testt\images\17_jpg.rf.5476395c13390c4de151ea6d3c77a7da.jpg: 320x512 51 manzanass, 14.0ms
image 4/4 c:\Users\JORGE\Documents\ProyectoIntegrador\Block-Detection\Yolo\custom_object_detector_yolo11_v2-1\testt\images\4_jpg.rf.9dbf89dd9fa7999aef4d7745b9865bde.jpg: 320x512 59 manzanass, 10.0ms
Speed: 1.7ms preprocess, 13.5ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 512)


In [53]:
# VISUALIZAR LOS RESULTADOS DE LAS DETECCIONES
res[0].show()

In [6]:
# VISUALIZAR LOS RESULTADOS DE LAS DETECCIONES
res[1].show()

IndexError: list index out of range